<a href="https://colab.research.google.com/github/MDKate/CBR/blob/main/%D0%A7%D0%91%D0%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>